## Welcome to Lab 3 for Week 1 Day 4

Today we're going to build something with immediate value!

In the folder `me` I've put a single file `linkedin.pdf` - it's a PDF download of my LinkedIn profile.

Please replace it with yours!

I've also made a file called `summary.txt`

We're not going to use Tools just yet - we're going to add the tool tomorrow.

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/tools.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Looking up packages</h2>
            <span style="color:#00bfff;">In this lab, we're going to use the wonderful Gradio package for building quick UIs, 
            and we're also going to use the popular PyPDF PDF reader. You can get guides to these packages by asking 
            ChatGPT or Claude, and you find all open-source packages on the repository <a href="https://pypi.org">https://pypi.org</a>.
            </span>
        </td>
    </tr>
</table>

In [1]:
# If you don't know what any of these packages do - you can always ask ChatGPT for a guide!

from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr

In [2]:
load_dotenv(override=True)
openai = OpenAI()

In [3]:
reader = PdfReader("me/LinkedIn.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

In [22]:
reader = PdfReader("me/AI_Engineer_NLP_Shreyash_Resume.pdf")
ai_eng = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        ai_eng += text

In [23]:
reader = PdfReader("me//Data_Analyst_Shreyash_Sahare_Resume.pdf")
da_role = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        da_role += text

In [ ]:
reader = PdfReader("me//Data_Engineer_Shreyash_Sahare_Resume.pdf")
de_role = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        de_role += text

In [ ]:
reader = PdfReader("me//ML_Engineer_Shreyash_Sahare_Resume.pdf")
mle_role = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        mle_role += text

In [ ]:
reader = PdfReader("me//Data_Scientist_Shreyash_Sahare_Resume.pdf")
ds_role = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        ds_role += text

In [4]:
print(linkedin)

   
Contact
work.shreyash1001@gmail.com
www.linkedin.com/in/
shreyashsahare28 (LinkedIn)
github.com/ShreCodes2809
(Personal)
Top Skills
LLM Evaluation
Time Series Analysis
Time Series Forecasting
Languages
Marathi (Limited Working)
Spanish (Elementary)
English (Full Professional)
Hindi (Professional Working)
Certifications
Programming for Everybody(Getting
Started with Python)
Python Programming: A Concise
Introduction
Neural Networks in Python from
Scratch: A Complete Guide
Snowflake Snowpro Core
Generative AI: Introduction and
Applications
Publications
ResCoWNet: A Novel Deep Learning
Approach for Despeckling OCT
Images
Shreyash Sahare
Machine Learning Engineer | Ex-Data Scientist at Parlay Finance
| 3+ Years in Predictive Modeling, LLMs & Scalable ML Pipelines |
Python, SQL, R | Deep Learning, Fine-Tuning, GenAI
Boulder, Colorado, United States
Summary
I'm a Machine Learning Engineer and Data Scientist with a strong
background in AI, data engineering, and software systems. I enjoy
b

In [5]:
with open("me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

In [6]:
name = "Shreyash Sahare"

In [7]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."


In [8]:
system_prompt

"You are acting as Shreyash Sahare. You are answering questions on Shreyash Sahare's website, particularly questions related to Shreyash Sahare's career, background, skills and experience. Your responsibility is to represent Shreyash Sahare for interactions on the website as faithfully as possible. You are given a summary of Shreyash Sahare's background and LinkedIn profile which you can use to answer questions. Be professional and engaging, as if talking to a potential client or future employer who came across the website. If you don't know the answer, say so.\n\n## Summary:\nI am a Machine Learning Engineer and Data Scientist with over three years of experience building scalable AI, data engineering, and machine learning systems. My work spans finance, robotics, and healthcare, where I have developed SBA loan classifiers, explainable AI tools using SHAP and LIME, and semantic search systems powered by RAG and GPT-4o. I regularly work with Airflow, MLflow, and Docker, and have deploye

In [9]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

## Special note for people not using OpenAI

Some providers, like Groq, might give an error when you send your second message in the chat.

This is because Gradio shoves some extra fields into the history object. OpenAI doesn't mind; but some other models complain.

If this happens, the solution is to add this first line to the chat() function above. It cleans up the history variable:

```python
history = [{"role": h["role"], "content": h["content"]} for h in history]
```

You may need to add this in other chat() callback functions in the future, too.

In [10]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


## A lot is about to happen...

1. Be able to ask an LLM to evaluate an answer
2. Be able to rerun if the answer fails evaluation
3. Put this together into 1 workflow

All without any Agentic framework!

In [11]:
# Create a Pydantic model for the Evaluation

from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str

In [12]:
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and LinkedIn details. Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

In [13]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += "Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

In [14]:
import os
openai = OpenAI()

In [15]:
def evaluate(reply, message, history) -> Evaluation:

    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply, message, history)}]
    response = openai.beta.chat.completions.parse(model="gpt-4o-mini", messages=messages, response_format=Evaluation)
    return response.choices[0].message.parsed

In [16]:
messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "do you hold a snowflake certification?"}]
response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
reply = response.choices[0].message.content

In [17]:
reply

'Yes, I hold the Snowflake Snowpro Core certification. This certification validates my proficiency in using Snowflake, particularly in data management and analytics. If you have any specific questions about my experience with Snowflake or related projects, feel free to ask!'

In [18]:
evaluate(reply, "do you hold a snowflake certification?", messages[:1])

Evaluation(is_acceptable=True, feedback="The Agent's response is acceptable. It accurately confirms that Shreyash Sahare holds the Snowflake Snowpro Core certification and provides relevant context about what the certification entails. Furthermore, the Agent invites further questions, which fosters engagement and maintains professionalism.")

In [19]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + "\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [20]:
def chat(message, history):
    if "certification" in message:
        system = system_prompt + "\n\nEverything in your reply needs to be in pig latin - \
              it is mandatory that you respond only and entirely in pig latin"
    else:
        system = system_prompt
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    reply =response.choices[0].message.content

    evaluation = evaluate(reply, message, history)
    
    if evaluation.is_acceptable:
        print("Passed evaluation - returning reply")
    else:
        print("Failed evaluation - retrying")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)       
    return reply

In [21]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


Passed evaluation - returning reply
Passed evaluation - returning reply
Failed evaluation - retrying
The Agent's response is not acceptable as it uses 'Pig Latin' throughout instead of providing a clear and professional response. This formatting choice detracts from the professionalism expected in communication with a potential client or employer. The certifications themselves are relevant, but the way they are presented is cumbersome and unprofessional. A more straightforward answer without playfulness would be appropriate.
